In [ ]:
import pandas as pd

df = pd.read_csv('./housePricing.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


# Process data

## replace columns with Nan and drop columns with multiple Nan

In [ ]:
# count the number of missing values of each column
nan = list(df.isna().sum())

# find the columns' names which need to be replaced or dropped
replaced_col_text = []
replaced_col_number = []
dropped_col = []

for idx, name in enumerate(df.columns):
    if nan[idx] == 0:
        continue
    if nan[idx] > len(df) * 0.4:
        dropped_col.append(name)
    # find text columns which have nan value inside
    elif df[name].dtype == object:
        replaced_col_text.append(name)
    # find digit columns which have nan value
    else: replaced_col_number.append(name)

print("text columns with nan:", replaced_col_text)
print("number columns with nan:", replaced_col_number)
print("dropping columns:", dropped_col)

# replace missing value of a text column with 'unknown'
df[replaced_col_text] = df[replaced_col_text].fillna(value='unknown')
# # replace missing value of a digit column with its column average
df[replaced_col_number] = df[replaced_col_number].fillna(value=df[replaced_col_number].mean())
# drop columns with more than 40% missing value
df_dropped = df.drop(columns=dropped_col, axis=1)

# check if any column has missing value
for col in df_dropped.columns:
    if df_dropped[col].isna().any():
        print(col, "has na value")

text columns with nan: ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
number columns with nan: ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
dropping columns: ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']


## convert text columns into numbers

In [ ]:
from sklearn.preprocessing import OneHotEncoder

text_columns = df_dropped.select_dtypes(include=['object']).columns
# apply one-hot encoding on text columns
one_hot = pd.get_dummies(df_dropped[text_columns])
# # Drop the original text columns and add numeric ones
new_df = pd.concat([df_dropped.drop(text_columns, axis=1), one_hot], axis=1)
new_df

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,0,0,1,0,0,0,0,1,0
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,0,0,0,1,0,0,0,0,1,0
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,0,0,1,0,0,0,0,1,0
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,0,0,0,1,0,0,0,0,1,0


## Normalize features and price

In [ ]:
from sklearn.preprocessing import StandardScaler

X = new_df.drop('SalePrice', axis=1)  # features
y = new_df['SalePrice']  # labels

numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])
scaler = StandardScaler()
y = scaler.fit_transform(y.values.reshape(-1, 1)).ravel()

# Select K best features

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

k_best = 20
selector = SelectKBest(score_func=f_regression, k=k_best)
X_selected = selector.fit_transform(X, y)

# Split data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X_selected, y, test_size=0.3, random_state=4)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=4)

print("train features shape:", X_train.shape)
print("train labels shape:", y_train.shape)

train features shape: (1022, 20)
train labels shape: (1022,)


# Support Vector Regression

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# SVM Regression model
model = SVR()
model.fit(X_train, y_train)

# Make predictions on training, validation, and test sets
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)
test_pred = model.predict(X_test)

# calcualate loss on each split of data
loss_train = mean_squared_error(y_train, train_pred)
loss_val = mean_squared_error(y_val, val_pred)
loss_test = mean_squared_error(y_test, test_pred)

print('Loss on train data:', "{:.3f}".format(loss_train))
print('Loss on val data:', "{:.3f}".format(loss_val))
print('Loss on test data:', "{:.3f}".format(loss_test))

Loss on train data: 0.129
Loss on val data: 0.090
Loss on test data: 0.136
